# In vitro T1

## EPC cell line 

In [1]:
import gzip, time
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import seaborn as sns
from Bio.Seq import Seq
from collections import Counter
import os

In [2]:
def get_innerBC(read_line,innerBC_ref):
    innerBC = read_line[0:6]
    num_Ns = sum([c=='N' for c in innerBC])
    #innerBC_ref = ['GTAGTC']
    if num_Ns > 1: return None 
    elif num_Ns == 1 and np.any([innerBC.replace('N',c) in innerBC_ref for c in 'ACTG']): 
        return [innerBC.replace('N',c) for c in 'ACTG' if innerBC.replace('N',c) in innerBC_ref ][0]
    elif innerBC in innerBC_ref: return innerBC
    else: return None
    
    
def get_innerBC_reads_count(sample_paths,innerBC_ref):
    innerBC_reads_count = {}
    innerBC_reads_count['r1'] = {}
    innerBC_reads_count['r2'] = {}

    for sample,paths in sample_paths.items():
        for fastq_path in paths['target_gene']:
            R1 = gzip.open(fastq_path.format('R1'))
            R2 = gzip.open(fastq_path.format('R2'))
            counter = 0
            start_time = time.time()
            while True:
                counter += 1
                if counter % 1000000 == 0: print(fastq_path+ ': Processed {} reads in {} seconds'.format(counter, time.time()-start_time))
                try:
                    r1_line = R1.readline().decode('utf-8')
                    r2_line = R2.readline().decode('utf-8')
                except:
                    print('ERROR extracting {}'.format(fastq_path))
                    break
                if r2_line == '' : break
                if r2_line[0] == '@': 
                    read2_name = r2_line
                    read1_name = r1_line

                    read1_seq = R1.readline().decode('utf-8')
                    read2_seq = R2.readline().decode('utf-8')

                    if get_innerBC(read2_seq,innerBC_ref)!=None and get_innerBC(read1_seq,innerBC_ref)!=None and get_innerBC(read2_seq,innerBC_ref)==get_innerBC(read1_seq,innerBC_ref):
                        innerBC = get_innerBC(read2_seq,innerBC_ref)
                        read2_QI = R2.readline().decode('utf-8')
                        read2_baseQ = R2.readline().decode('utf-8')
                        read2 = [read2_name , read2_seq , read2_QI , read2_baseQ]

                        read1_QI = R1.readline().decode('utf-8')
                        read1_baseQ = R1.readline().decode('utf-8')
                        read1 = [read1_name , read1_seq , read1_QI , read1_baseQ]

                        #print(read1)

                        if innerBC in innerBC_reads_count['r1']:
                            innerBC_reads_count['r1'][innerBC] += read1
                            innerBC_reads_count['r2'][innerBC] += read2
                        else:
                            innerBC_reads_count['r1'][innerBC] = read1
                            innerBC_reads_count['r2'][innerBC] = read2

                    else:
                        read2_QI = R2.readline().decode('utf-8')
                        read2_baseQ = R2.readline().decode('utf-8')
                        read2 = [read2_name , read2_seq , read2_QI , read2_baseQ]

                        read1_QI = R1.readline().decode('utf-8')
                        read1_baseQ = R1.readline().decode('utf-8')
                        read1 = [read1_name , read1_seq , read1_QI , read1_baseQ]

                        if 'unclassified' in innerBC_reads_count['r1']:
                            innerBC_reads_count['r1']['unclassified'] += read1
                            innerBC_reads_count['r2']['unclassified'] += read2
                        else:
                            innerBC_reads_count['r1']['unclassified'] = read1
                            innerBC_reads_count['r2']['unclassified'] = read2
    
    return innerBC_reads_count

In [26]:
# R701
sample_paths = {
    'T1_in_vitro': {
        'target_gene':[
                       '/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R701/SQ24024299-YJH-cas9-61-R701_combined_{}.fastq.gz',
                       ]
          }
                }
innerBC_ref = ['GTAGTC']
innerBC_reads_count = get_innerBC_reads_count(sample_paths,innerBC_ref)

os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R701/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R701'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R701/*.fq|xargs -I {} gzip {}')



/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R701/SQ24024299-YJH-cas9-61-R701_combined_{}.fastq.gz: Processed 1000000 reads in 10.085916757583618 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R701/SQ24024299-YJH-cas9-61-R701_combined_{}.fastq.gz: Processed 2000000 reads in 20.31579852104187 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R701/SQ24024299-YJH-cas9-61-R701_combined_{}.fastq.gz: Processed 3000000 reads in 30.777148962020874 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R701/SQ24024299-YJH-cas9-61-R701_combined_{}.fastq.gz: Processed 4000000 reads in 40.999396562576294 seconds


rm: cannot remove ‘/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R701/*.fq.gz’: No such file or directory


0

In [27]:
# R709
sample_paths = {
    'T1_in_vitro': {
        'target_gene':[
                       '/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R709/SQ24024299-YJH-cas9-61-R709_combined_{}.fastq.gz',
                       ]
          }
                }
innerBC_ref = ['GTAGTC','CTTGTA','CATCGA','GACCTA']
innerBC_reads_count = get_innerBC_reads_count(sample_paths,innerBC_ref)

os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R709/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R709'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R709/*.fq|xargs -I {} gzip {}')



/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R709/SQ24024299-YJH-cas9-61-R709_combined_{}.fastq.gz: Processed 1000000 reads in 10.336155652999878 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R709/SQ24024299-YJH-cas9-61-R709_combined_{}.fastq.gz: Processed 2000000 reads in 20.42610764503479 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R709/SQ24024299-YJH-cas9-61-R709_combined_{}.fastq.gz: Processed 3000000 reads in 30.494313955307007 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R709/SQ24024299-YJH-cas9-61-R709_combined_{}.fastq.gz: Processed 4000000 reads in 40.62560248374939 seconds
/syn1/liangzhe

rm: cannot remove ‘/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R709/*.fq.gz’: No such file or directory


0

In [28]:
# R705
sample_paths = {
    'T1_in_vitro': {
        'target_gene':[
                       '/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz',
                       ]
          }
                }
innerBC_ref = ['ACGTAG','CAGATC','CTTGTA','TTAGGC','GGCTAC','CTGTGA','GTAGTC','GATCAG','GACCTA','ATCACG','TGCACT','GCACTA','CATCGA','TAGCTT','ACCAGT','CGATGT','GATGCA','TGTAGC']
innerBC_reads_count = get_innerBC_reads_count(sample_paths,innerBC_ref)

os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R705/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R705'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R705/*.fq|xargs -I {} gzip {}')


/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 1000000 reads in 11.362395286560059 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 2000000 reads in 22.83995294570923 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 3000000 reads in 34.78624391555786 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 4000000 reads in 47.20223379135132 seconds
/syn1/liangzhen

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 35000000 reads in 430.22565746307373 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 36000000 reads in 443.3189814090729 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 37000000 reads in 456.65142488479614 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 38000000 reads in 469.30547285079956 seconds
/syn1/lia

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 69000000 reads in 865.0164184570312 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 70000000 reads in 878.0960674285889 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 71000000 reads in 890.9333028793335 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-051621/Sample_SQ24024299-YJH-cas9-61-R705/SQ24024299-YJH-cas9-61-R705_combined_{}.fastq.gz: Processed 72000000 reads in 903.9301309585571 seconds
/syn1/liangz

rm: cannot remove ‘/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/EPC/R705/*.fq.gz’: No such file or directory


0

## 3T3 cell line

In [29]:
# R701
sample_paths = {
    'T1_in_vitro': {
        'target_gene':[
                       '/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R701/SQ24024300-YJH-cas9-62-R701_combined_{}.fastq.gz',
                       ]
          }
                }
innerBC_ref = ['GTAGTC']
innerBC_reads_count = get_innerBC_reads_count(sample_paths,innerBC_ref)

os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R701/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R701'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R701/*.fq|xargs -I {} gzip {}')



/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R701/SQ24024300-YJH-cas9-62-R701_combined_{}.fastq.gz: Processed 1000000 reads in 11.078634977340698 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R701/SQ24024300-YJH-cas9-62-R701_combined_{}.fastq.gz: Processed 2000000 reads in 21.584100008010864 seconds


rm: cannot remove ‘/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R701/*.fq.gz’: No such file or directory


0

In [30]:
# R709
sample_paths = {
    'T1_in_vitro': {
        'target_gene':[
                       '/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R709/SQ24024300-YJH-cas9-62-R709_combined_{}.fastq.gz',
                       ]
          }
                }
innerBC_ref = ['GTAGTC','CTTGTA','CATCGA','GACCTA']
innerBC_reads_count = get_innerBC_reads_count(sample_paths,innerBC_ref)

os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R709/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R709'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R709/*.fq|xargs -I {} gzip {}')



/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R709/SQ24024300-YJH-cas9-62-R709_combined_{}.fastq.gz: Processed 1000000 reads in 10.06537675857544 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R709/SQ24024300-YJH-cas9-62-R709_combined_{}.fastq.gz: Processed 2000000 reads in 19.865294218063354 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R709/SQ24024300-YJH-cas9-62-R709_combined_{}.fastq.gz: Processed 3000000 reads in 29.649571657180786 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R709/SQ24024300-YJH-cas9-62-R709_combined_{}.fastq.gz: Processed 4000000 reads in 39.3673939704895 seconds
/syn1/liangzhen

rm: cannot remove ‘/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R709/*.fq.gz’: No such file or directory


0

In [31]:
# R705
sample_paths = {
    'T1_in_vitro': {
        'target_gene':[
                       '/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz',
                       ]
          }
                }
innerBC_ref = ['ACGTAG','CAGATC','CTTGTA','TTAGGC','GGCTAC','CTGTGA','GTAGTC','GATCAG','GACCTA','ATCACG','TGCACT','GCACTA','CATCGA','TAGCTT','ACCAGT','CGATGT','GATGCA','TGTAGC']
innerBC_reads_count = get_innerBC_reads_count(sample_paths,innerBC_ref)

os.system('rm /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R705/*.fq.gz')
for fq,pooled_reads in innerBC_reads_count.items():
    for innerBC,reads_set in pooled_reads.items():
        #print(innerBC+fq)
        with open('/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R705'+'/'+innerBC+'_'+fq+'.fq','w+') as f:
            f.writelines(reads_set)
os.system('ls /syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R705/*.fq|xargs -I {} gzip {}')


/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 1000000 reads in 11.287111043930054 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 2000000 reads in 22.49531865119934 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 3000000 reads in 33.7679922580719 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 4000000 reads in 45.02293515205383 seconds
/syn1/liangzhen/

/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 35000000 reads in 393.01869559288025 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 36000000 reads in 404.25015473365784 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 37000000 reads in 415.44397163391113 seconds
/syn1/liangzhen/jinhua_jilab_project/data/DNA_Amplicon/plasmid/s2164g01113_MCS-20240328-L-04-2024-04-061800/Sample_SQ24024300-YJH-cas9-62-R705/SQ24024300-YJH-cas9-62-R705_combined_{}.fastq.gz: Processed 38000000 reads in 426.583580493927 seconds
/syn1/lian

rm: cannot remove ‘/syn1/liangzhen/jinhua_jilab_project/result/DNA_Amplicon/T1_in_vitro/3T3/R705/*.fq.gz’: No such file or directory


0